In [1]:
import sys; sys.path.append("../")
from architecture import Architecture
from cascade import CascadeClassifier
from feature_extractor import FeatureExtractor
import numpy as np
import time
import pickle as pkl
import torch

In [ ]:
parent = '../Features/'

In [2]:
torch.cuda.is_available()

True

In [3]:
X = np.load(parent + 'selected_features.npz')['arr_0']
y = np.load(parent + 'labels.npy')
print(X.shape)
print(y.shape)


(6396, 6977)
(6977,)


In [4]:
s_t = time.time()
architecture = Architecture(X, y, verbose=False, Ftarget=0.01, f=0.5, d=0.99, v_size=0.3)
# architecture = Architecture(X, y, verbose=True, Ftarget=0.01, f=0.4, d=0.9999, v_size=0.3)
cascadeClassifier = architecture.build()
e_t = time.time()
print(f"Time taken: {e_t - s_t}")

Building layer 1
Building layer 1, classifier 1
Starting to choose classifier
Finished training weak classifier 1 / 1
False positive rate: 1.0, required false positive rate: 0.4
Detection rate: 1.0, required detection rate: 0.9999
Building layer 1, classifier 2
Starting to choose classifier
Finished training weak classifier 2 / 2
False positive rate: 1.0, required false positive rate: 0.4
Detection rate: 1.0, required detection rate: 0.9999
Building layer 1, classifier 3
Starting to choose classifier
Finished training weak classifier 3 / 3
False positive rate: 1.0, required false positive rate: 0.4
Detection rate: 1.0, required detection rate: 0.9999
Building layer 1, classifier 4
Starting to choose classifier
Finished training weak classifier 4 / 4
False positive rate: 1.0, required false positive rate: 0.4
Detection rate: 1.0, required detection rate: 0.9999
Building layer 1, classifier 5
Starting to choose classifier
Finished training weak classifier 5 / 5
False positive rate: 0.693

In [5]:
layers = []
for SC in cascadeClassifier.strong_classifiers:
    layers.append(len(SC.weak_classifiers))
print(layers)

[11, 82, 200]


In [6]:
t_labels = y
predictions = cascadeClassifier.predict(X)
print(np.sum(predictions == 1), end=' / ')
print(np.sum(t_labels == 1))
print(np.sum(predictions == 0), end=' / ')
print(np.sum(t_labels == 0))
# accuracy:
acc = np.sum(predictions == t_labels) / len(t_labels)
print('acc = ', acc)
# precision:
prec = np.sum(predictions[t_labels == 1] == 1) / np.sum(predictions == 1)
print('prec = ', prec)
# recall:
rec = np.sum(predictions[t_labels == 1] == 1) / np.sum(t_labels == 1)
print('rec = ', rec)
# f1 score:
f1 = 2 * prec * rec / (prec + rec)
print('f1 = ', f1)


2456 / 2429
4521 / 4548
acc =  0.9961301418947972
prec =  0.9890065146579805
rec =  1.0
f1 =  0.9944728761514842


In [7]:
feature_extractor = FeatureExtractor(shape=(19,19),
                                     percentile=10,
                                     all_features_file=parent + 'all_features.npz',
                                     selected_features_file=parent + 'selected_features.npz',
                                     labels_file=parent + 'labels.npy',
                                     indecies_file=parent + 'indecies.npy')

In [8]:
indecies, X = feature_extractor.selectPercentile()
y = np.load(parent + 'labels.npy')
print(X.shape, y.shape)

(6396, 6977) (6977,)


In [9]:
s_t = time.time()
t_f_idx_map, t_features, t_labels = feature_extractor.extractFeaturesByIndecies(pos_path='face.test/test/face/',
                                            neg_path='face.test/test/non-face/',
                                            cascadeClassifier=cascadeClassifier)
print(t_features.shape, t_labels.shape)
predictions = cascadeClassifier.predict(t_features, t_f_idx_map)
print('test accuracy: ', np.sum(predictions == t_labels) / t_labels.shape[0])
# precision:
prec = np.sum(predictions[t_labels == 1] == 1) / np.sum(predictions == 1)
print('prec = ', prec)
# recall:
rec = np.sum(predictions[t_labels == 1] == 1) / np.sum(t_labels == 1)
print('rec = ', rec)
# f1 score:
f1 = 2 * prec * rec / (prec + rec)
print('f1 = ', f1)
print('time taken: ', time.time() - s_t, 's')

Now reading dataset...
Now extracting features from dataset...
(228, 24045) (24045,)
test accuracy:  0.9731752963194011
prec =  0.27877237851662406
rec =  0.2309322033898305
f1 =  0.2526071842410197
time taken:  0.672966718673706 s
